In [1]:
import uuid
import json
import redis
import paho.mqtt.client as mqtt 

#connect to a broker to send/receive messages to/from the broker
client = mqtt.Client()


series1 = "ambulance"


REDIS_HOST = "redis-17637.c114.us-east-1-4.ec2.cloud.redislabs.com"
REDIS_PORT = 17637
REDIS_USER = "default"
REDIS_PASSWORD = "fFh38GR6BxHI0xVoGbsUKAzPZlCBXLAA"

redis_client = redis.Redis(host = REDIS_HOST, port = REDIS_PORT, password = REDIS_PASSWORD, username= REDIS_USER)
print("Is connected:", redis_client.ping())


Is connected: True


In [2]:
def check_series_creation(redis_client, mac_address):
    '''checks for the existence of the series and creates them'''
    try:
        redis_client.ts().create(mac_address+":ambulance", label = {"ambulance" } )
    except redis.ResponseError:
        pass
    

def on_connect(client, userdata, flags, rc):
    #subscribe to a topic
    client.subscribe('s291871')


def on_message(client, userdata, msg):
    message = msg.payload.decode() 
    message_dict = json.loads(message)
    print(message)
    #whenever a new mac_address is received, we need to create the redis timeseries
    check_series_creation(redis_client, message_dict["mac_address"])
    
    #store the data
    redis_client.ts().add(message_dict["mac_address"]+":ambulance", message_dict["timestamp"], message_dict["ambulance"])
    


#bind the on_connect and on_message method to the client
client.on_connect = on_connect
client.on_message = on_message

#connect to the broker 
client.connect('mqtt.eclipseprojects.io', 1883)

0

In [3]:
#keep listening over the topic we subscribed to using an infinite loop 
client.loop_forever()

{"mac_address": "0xc8d200151708", "timestamp": 1678719792628, "ambulance": 1}
{"mac_address": "0xc8d200151708", "timestamp": 1678719795527, "ambulance": 1}
{"mac_address": "0xc8d200151708", "timestamp": 1678719798556, "ambulance": 1}
{"mac_address": "0xc8d200151708", "timestamp": 1678719801668, "ambulance": 1}
{"mac_address": "0xc8d200151708", "timestamp": 1678719861725, "ambulance": 0}
{"mac_address": "0xc8d200151708", "timestamp": 1678719864669, "ambulance": 0}
{"mac_address": "0xec1c6e72a03d", "timestamp": 1678719865850, "ambulance": 0}
{"mac_address": "0xc8d200151708", "timestamp": 1678719867720, "ambulance": 0}
{"mac_address": "0xec1c6e72a03d", "timestamp": 1678719868709, "ambulance": 0}
{"mac_address": "0xc8d200151708", "timestamp": 1678719870519, "ambulance": 0}
{"mac_address": "0xec1c6e72a03d", "timestamp": 1678719871666, "ambulance": 0}
{"mac_address": "0xec1c6e72a03d", "timestamp": 1678719875006, "ambulance": 0}
{"mac_address": "0xc8d200151708", "timestamp": 1678719876311, "a

KeyboardInterrupt: 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=957cfe07-d816-4770-9cc0-d7c1b4767886' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>